In [1]:
import pandas as pd
from sklearn import metrics
import numpy as np

In [2]:
df = pd.read_excel('sdbi_proxy_bad_def_dataset.xlsx')
print(df.shape)
df = df[df['mob']<=24]
print(df.shape)

(172996, 3)
(172929, 3)


In [3]:
df['max_mob'] = df.groupby(['id'])['mob'].transform(max)

C:\Users\siddh\AppData\Local\Temp\ipykernel_16932\3371001165.py:1: FutureWarning: The provided callable <built-in function max> is currently using SeriesGroupBy.max. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "max" instead.
  df['max_mob'] = df.groupby(['id'])['mob'].transform(max)


In [4]:
print(df.shape)
df = df[~(df['max_mob'] < 24)]
print(df.shape)


(172929, 4)
(172927, 4)


In [5]:
df

,id,mob,dpd,max_mob
0,1,0,0,24
1,1,1,0,24
2,1,2,0,24
3,1,3,0,24
4,1,4,0,24
...,...,...,...,...
172991,6999,20,41,24
172992,6999,21,71,24
172993,6999,22,0,24
172994,6999,23,0,24


In [6]:
print(df.shape)
df = df.drop_duplicates(subset=['id','mob'])
print(df.shape)

(172927, 4)
(172927, 4)


We need to create

-- max_dpd column, which is ,aximum dpd for every customer within first 24 months

-- using max_dpd create 'eventual_bad' which 1 if >=90 else 0

In [7]:
o_base = pd.pivot_table(df,index = 'id', values = 'dpd', aggfunc=  'max').reset_index()

In [8]:
o_base.head()

,id,dpd
0,1,86
1,2,363
2,3,0
3,4,0
4,5,0


In [9]:
# new_df = pd.DataFrame(data = o_base['id'], columns = ['id'])

In [10]:
# o_base['dpd'] > 90

In [11]:
# o_base[o_base['dpd'] > 90]

In [12]:
# new_df['24m_90'] = o_base['dpd'] > 90

In [13]:
# new_df['24m_30'] =  new_df['24m_30'].map(  {False:0,True:1})
# new_df['24m_60'] = new_df['24m_60'].map(  {False:0,True:1})
# new_df['24m_90'] = new_df['24m_90'].map(  {False:0,True:1})
# new_df['24m_120'] = new_df['24m_120'].map(  {False:0,True:1})
# new_df['24m_150'] = new_df['24m_150'].map(  {False:0,True:1})
# -----------
#new_df['24m_90'] = new_df['24m_90'].map(  {False:0,True:1})

In [14]:
# df_12 = df[df['mob']<=12] 
# new_df_12 = pd.pivot_table(df_12,index = 'id', values = 'dpd', aggfunc=  'max').reset_index()


# new_df['{}m_{}'.format(12,60)] = np.where(new_df_12['dpd'] >= 60,1,0)

In [15]:
# new_df_12

In [16]:
# t = df[df['mob']<12]

# o_base = pd.pivot_table(t,index = 'id', values = 'dpd', aggfunc=  'max').reset_index()

# o_base

In [17]:
o_base['eventual_bad'] = np.where((o_base['dpd'] >90), 1 , 0)

In [18]:
o_base.head()

,id,dpd,eventual_bad
0,1,86,0
1,2,363,1
2,3,0,0
3,4,0,0
4,5,0,0


In [19]:
m = 12

d = 60

t = df[df['mob']<m]

t = pd.pivot_table(t, index = 'id',values = 'dpd', aggfunc='max').reset_index()

t['proxy_bad'] = np.where((t['dpd']>d), 1 , 0).astype(int)

## Create a common dataframe
temp = pd.merge(o_base, t[['id','proxy_bad']], how='left',on='id')



# dp = {}

# dp['name'] = '{m}m_{d}'.format()
# dp['count'] = len(temp)
# do['bad_count'] = len(temp[temp['proxy_bad'] ==1])
temp[temp['proxy_bad']>0].head()

,id,dpd,eventual_bad,proxy_bad
12,13,105,1,1
110,111,75,0,1
274,275,75,0,1
356,357,105,1,1
383,384,116,1,1


In [20]:
pd.crosstab(temp['proxy_bad'], temp['eventual_bad'])

eventual_bad,0,1
proxy_bad,,
0,6701,147
1,30,119


In [21]:
o_base.head()

,id,dpd,eventual_bad
0,1,86,0
1,2,363,1
2,3,0,0
3,4,0,0
4,5,0,0


**Lets build a loop**

In [40]:
# mob = [6,12,15,18,20]
mob = [6,9,12,15,18]

dpd = [30,45,60,75,90,120]

Bad_df = pd.DataFrame()

for m in mob:
    for d in dpd:
        t = df[df['mob']<m]
        
        t = pd.pivot_table(t, index = 'id',values = 'dpd', aggfunc='max').reset_index()
        
        t['proxy_bad'] = np.where((t['dpd']>d), 1 , 0)
        
        ## Create a common dataframe
        temp = pd.merge(o_base, t[['id','proxy_bad']], how='left',on='id')
        
        
        dic = {}
        
        dic['name'] = '{}m_{}'.format(m,d)
        dic['count'] = len(temp)
        dic['eventual_bad'] = len(temp[temp['eventual_bad'] == 1])
        dic['proxy_bad'] = len(temp[temp['proxy_bad'] ==1])
        dic['accuracy'] = metrics.accuracy_score(temp['eventual_bad'], temp['proxy_bad'])
        dic['precision'] = metrics.precision_score(temp['eventual_bad'], temp['proxy_bad'])
        dic['recall'] = metrics.recall_score(temp['eventual_bad'], temp['proxy_bad'])
        dic['f1_score'] = metrics.f1_score(temp['eventual_bad'], temp['proxy_bad'])
        
        bad_df = pd.DataFrame([dic])
        
        Bad_df = pd.concat([Bad_df,bad_df], axis = 0)
        
        
        
        
        
        

In [41]:
Bad_df.sort_values(by = 'f1_score',ascending=False)

,name,count,eventual_bad,proxy_bad,accuracy,precision,recall,f1_score
0,18m_90,6997,266,216,0.992854,1.000000,0.812030,0.896266
0,18m_75,6997,266,245,0.988709,0.881633,0.812030,0.845401
0,18m_60,6997,266,305,0.980134,0.708197,0.812030,0.756567
0,15m_90,6997,266,142,0.982278,1.000000,0.533835,0.696078
0,15m_75,6997,266,152,0.980849,0.934211,0.533835,0.679426
0,18m_45,6997,266,374,0.970559,0.580214,0.815789,0.678125
0,15m_60,6997,266,178,0.978848,0.831461,0.556391,0.666667
0,18m_30,6997,266,395,0.967558,0.549367,0.815789,0.656581
0,15m_45,6997,266,287,0.969558,0.592334,0.639098,0.614828
0,15m_30,6997,266,309,0.966700,0.553398,0.642857,0.594783


In [46]:
Bad_df['weighted_avg'] =((Bad_df['proxy_bad']/Bad_df['eventual_bad']) + (Bad_df['f1_score']*3) + (bad_df['recall']*2) + (bad_df['precision']*2)+(bad_df['accuracy']*1)) / 9

In [47]:
Bad_df.sort_values(by = 'weighted_avg',ascending=False)

,name,count,eventual_bad,proxy_bad,accuracy,precision,recall,f1_score,weighted_avg
0,18m_90,6997,266,216,0.992854,1.000000,0.812030,0.896266,0.744480
0,18m_75,6997,266,245,0.988709,0.881633,0.812030,0.845401,0.739639
0,18m_30,6997,266,395,0.967558,0.549367,0.815789,0.656581,0.739356
0,18m_45,6997,266,374,0.970559,0.580214,0.815789,0.678125,0.737765
0,18m_60,6997,266,305,0.980134,0.708197,0.812030,0.756567,0.735091
0,15m_30,6997,266,309,0.966700,0.553398,0.642857,0.594783,0.682833
0,15m_45,6997,266,287,0.969558,0.592334,0.639098,0.614828,0.680325
0,15m_60,6997,266,178,0.978848,0.831461,0.556391,0.666667,0.652074
0,15m_90,6997,266,142,0.982278,1.000000,0.533835,0.696078,0.646841
0,15m_75,6997,266,152,0.980849,0.934211,0.533835,0.679426,0.645467
